In [1]:
pip install pandas numpy networkx matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# 100 tane anlamlı ürün ismi
product_names = [
    'Wireless Mouse', 'Bluetooth Headphones', 'Laptop Stand', 'Portable Charger', 'USB-C Hub', 'Smartwatch', 'Noise Cancelling Headphones', 
    'Fitness Tracker', 'External Hard Drive', 'LED Desk Lamp', 'Ergonomic Chair', 'Mechanical Keyboard', 'Gaming Mouse', '4K Monitor', 
    'Smartphone Stand', 'Wireless Earbuds', 'Bluetooth Speaker', 'Smart Home Hub', 'Robot Vacuum', 'Smart Light Bulb', 'Portable Projector', 
    'Webcam', 'Streaming Microphone', 'Graphics Tablet', 'Smart Thermostat', 'Electric Toothbrush', 'Digital Photo Frame', 'E-Reader', 
    'Smart Lock', 'Wireless Security Camera', 'Air Purifier', 'Electric Kettle', 'Espresso Machine', 'Blender', 'Slow Cooker', 'Rice Cooker', 
    'Air Fryer', 'Food Processor', 'Stand Mixer', 'Toaster Oven', 'Electric Griddle', 'Instant Pot', 'Electric Skillet', 'Cordless Drill', 
    'Tool Set', 'Circular Saw', 'Electric Screwdriver', 'Lawn Mower', 'Leaf Blower', 'Pressure Washer', 'Garden Hose', 'Garden Shears', 
    'Smart Sprinkler', 'Fitness Bike', 'Treadmill', 'Yoga Mat', 'Dumbbell Set', 'Kettlebell', 'Resistance Bands', 'Foam Roller', 
    'Weight Bench', 'Jump Rope', 'Pull-Up Bar', 'Running Shoes', 'Hiking Boots', 'Tent', 'Sleeping Bag', 'Camping Stove', 'Backpack', 
    'Flashlight', 'Binoculars', 'Fishing Rod', 'Cooler', 'Inflatable Kayak', 'Paddle Board', 'Snorkel Set', 'Wet Suit', 'Life Jacket', 
    'Picnic Blanket', 'Portable Grill', 'Fire Pit', 'Outdoor Speaker', 'Hammock', 'Solar Charger', 'Water Bottle', 'Travel Mug', 
    'Luggage Set', 'Packing Cubes', 'Travel Pillow', 'Travel Adapter', 'Portable WiFi Hotspot', 'Noise-Cancelling Earplugs', 'Travel Backpack', 
    'Action Camera', 'Drone', 'Digital Camera', 'Camera Lens', 'Tripod', 'Memory Card', 'Camera Bag', 'Photo Printer'
]

# Mevcut veri setini yükleme
df_customers = pd.read_csv('data/olist_customers_dataset.csv')
df_order_items = pd.read_csv('data/olist_order_items_dataset.csv')
df_orders = pd.read_csv('data/olist_orders_dataset.csv')
df_products = pd.read_csv('data/olist_products_dataset.csv')

# Veri temizleme
df_orders.isnull().sum()
df_orders = df_orders.dropna()
df_order_items.isnull().sum()
df_products = df_products.dropna()

# Ürün ID'lerini alma
product_ids = df_products['product_id'].unique()

# Ürün ID'leri ile ürün isimlerini eşleştirme
np.random.seed(42)  # Rastgeleliğin tekrarlanabilir olması için
product_name_mapping = dict(zip(product_ids, np.random.choice(product_names, len(product_ids), replace=True)))

# Ürün isimlerini df_order_items veri setine ekleme
df_order_items['product_name'] = df_order_items['product_id'].map(product_name_mapping)

# Müşteri ve sipariş verilerini birleştirme
df_customers_orders = df_customers.merge(df_orders, on='customer_id')
orders_order_items = df_orders.merge(df_order_items, on='order_id')
top_customer_ids = orders_order_items['customer_id'].value_counts().head(20)
top_customer_data = orders_order_items[orders_order_items['customer_id'].isin(top_customer_ids.index)]

# İlk 100 satırı al
df_customers_subset = df_customers[df_customers['customer_id'].isin(top_customer_data['customer_id'].unique())]

# Graf oluşturalım
G = nx.DiGraph()

# Kullanıcıları düğümler olarak ekle
for _, row in df_customers_subset.iterrows():
    G.add_node(row['customer_unique_id'])

# Siparişleri ve ürünleri bağlantılar olarak ekle
for _, row in orders_order_items.iterrows():
    df_customers_ids = df_customers_subset.loc[df_customers_subset['customer_id'] == row['customer_id'], 'customer_unique_id'].values
    if len(df_customers_ids) > 0:
        G.add_edge(df_customers_ids[0], row['product_name'])

# Grafı çizelim
plt.figure(figsize=(10, 8))

# Müşteri ve ürün düğümlerini ayırt etmek için renkler
node_colors = ['lightblue' if node in df_customers_subset['customer_unique_id'].values else 'lightgreen' for node in G.nodes]

nx.draw_networkx(G, with_labels=True, node_color=node_colors, node_size=500, font_size=7)
plt.title("Kullanıcılar ve Aldıkları Ürünler Arasındaki İlişkiler")
plt.show()